In [2]:
from transformers import AutoModel, AutoTokenizer
import sys
sys.path.append("..")
from verl.workers.rollout.vllm_rollout import vLLMRollout
# Replace with your desired model name (from https://huggingface.co/models)
model_name = "emrecanacikgoz/Qwen2.5-7B-Instruct-ToolRL-grpo-cold"

# Download tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
# from /u/zluo8/sim2real/ToolRL/verl/trainer/main_ppo.py line 116
config = {
    'enforce_eager': True,
    'free_cache_engine': True,
    'prompt_length': 2048,
    'response_length': 1024,
    'dtype': 'bfloat16',
    'gpu_memory_utilization': 0.6,
    'load_format': 'dummy_dtensor',
}
rollout = vLLMRollout(actor_module=model, config=config)


WARNING 10-06 14:05:41 _core_ext.py:180] Failed to import from vllm._core_C with ImportError('/u/zluo8/sim2real/ToolRL/.venv/lib/python3.12/site-packages/vllm/_core_C.abi3.so: undefined symbol: _ZN5torch6detail10class_baseC2ERKSsS3_SsRKSt9type_infoS6_')


RuntimeError: Tried to instantiate class '_core_C.ScalarType', but it does not exist! Ensure that it is registered via torch::class_

In [ ]:
# load dataset

import pyarrow.parquet as pq

# Load the Parquet file
table = pq.read_table("../dataset/rlla_4k/test.parquet")

# Convert to a record batch (like a table of rows)
records = table.to_pylist()  # This is a list of dicts, one per row


In [ ]:

print("Model and tokenizer loaded successfully.")


for idx, record in enumerate(records[:3]):  # Display the first 3 records
    print(f'----- Record {idx} -----')
    prompt = record['prompt']
    for p in prompt:
        print("role: " + p['role'])
        print("content:")
        print(p['content'])
    # This is from /u/zluo8/sim2real/ToolRL/verl/utils/dataset/rl_dataset.py line 131
    prompt = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
    output = rollout.generate_sequences([prompt])
    print("Output:\n", output[0])

Prompt:
 <|im_start|>system
You are a helpful multi-turn dialogue assistant capable of leveraging tool calls to solve user tasks and provide structured chat responses.

**Available Tools**
In your response, you can use the following tools:
1. Name: GetPowerBINews
Description: Retrieve news related to Power BI.
Parameters: {"page": {"description": "The page number to retrieve.", "type": "int", "default": ""}}
2. Name: SingleOffer
Description: API that retrieves information for a single offer from Avito
Parameters: {"singleav": {"description": "The URL of the Avito offer to retrieve information for", "type": "string", "default": "https://www.avito.ma/fr/autre_secteur/maisons_et_villas/Villa_OCP_4_faces_sur_550_metre_de_terrain_49107436.htm"}}
3. Name: Domain WHOIS Lookup API
Description: Retrieves WHOIS information for a given domain name.
Parameters: {"domain_name": {"description": "The domain name for which you would like to retrieve WHOIS details.", "type": "string", "default": ""}}
4